# Analyse exploratoire des données Network

In [ ]:
# Librairies
import pandas as pd
from pickleshare import PickleShareDB

from sklearn.preprocessing import LabelEncoder

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.subplots as sp
from plotly.graph_objs import Bar
import plotly.express as px

## Chargement des données

Nous chargeons les données depuis le fichier des données préprarées.

In [ ]:
# Données nettoyées
db = PickleShareDB('../prep_data/kity')

df_net_1 = db['net_attack_1_clean']
df_net_2 = db['net_attack_2_clean']
df_net_3 = db['net_attack_3_clean']
df_net_4 = db['net_attack_4_clean']
df_net_norm = db['net_norm_clean']

# Données brutes
'''
df_net_1 = pd.read_csv('../datasets/Network datatset/csv/attack_1.csv')
df_net_2 = pd.read_csv('../datasets/Network datatset/csv/attack_2.csv')
df_net_3 = pd.read_csv('../datasets/Network datatset/csv/attack_3.csv')
df_net_4 = pd.read_csv('../datasets/Network datatset/csv/attack_4.csv')
df_net_norm = pd.read_csv('../datasets/Network datatset/csv/normal.csv')
'''

In [ ]:
# nombre de valeurs unique par colonne pour chaque dataset
nunique_1 = df_net_1.nunique()
nunique_2 = df_net_2.nunique()
nunique_3 = df_net_3.nunique()
nunique_4 = df_net_4.nunique()
nunique_norm = df_net_norm.nunique()
print(nunique_1)
print(nunique_2)
print(nunique_3)
print(nunique_4)
print(nunique_norm)

Nous commençons par observer le premier, comme ils doivent à priori avoir la même structure, pour comprendre les données que l'on a.

In [ ]:
df_net_1.info()

## Analyse des différents types de colonnes

In [ ]:
# Colonnes de chaque type
df_bool = df_net_1.select_dtypes(include='bool')
print("Colonnes de type booléen : \n", df_bool.columns,"\n")

df_obj = df_net_1.select_dtypes(include='object')
print("Colonnes de type objet : \n",df_obj.columns,"\n")

df_cat = df_net_1.select_dtypes(include='category')
print("Colonnes de type catégorie : \n", df_cat.columns,"\n")

df_num = df_net_1.select_dtypes(include='number')
print("Colonnes de type nombre : \n", df_num.columns,"\n")  

df_time = df_net_1.select_dtypes(include='datetime')
print("Colonnes de type datetime : \n", df_time.columns)

### Analyse colonnes booléennes

In [ ]:
fig = make_subplots(rows=1, cols=len(df_bool.columns), subplot_titles=df_bool.columns)

for i, col in enumerate(df_bool.columns):
    value_counts = df_bool[col].value_counts()
    fig.add_trace(go.Bar(x=value_counts.index, y=value_counts.values, name=col), row=1, col=i+1)


fig.update_layout(height=300, width=500, title="Fréquence des valeurs uniques par colonne (category)", showlegend=False)

fig.show()

### Analyse des colonnes numériques

In [ ]:
df_num.describe()

In [ ]:
# Statistiques descriptives
desc_stats = df_num.describe()
desc_stats = desc_stats.drop('count') # suppression de la colonne count pour la visualisation

fig = make_subplots(rows=1, cols=len(desc_stats.columns), subplot_titles=desc_stats.columns)
for i, col in enumerate(desc_stats.columns):
    fig.add_trace(go.Bar(x=desc_stats.index, y=desc_stats[col], name=col),row=1, col=i+1)

fig.update_layout(height=700, width=1200,title="Statistiques descriptives par colonne",showlegend=False)

fig.show()

In [ ]:
# Répartition des valeurs pour chaque colonne
for col in df_num.columns:
    print(col, df_num[col].value_counts())

In [ ]:
fig = make_subplots(rows=1, cols=len(df_cat.columns), subplot_titles=df_cat.columns)

for i, col in enumerate(df_cat.columns):
    value_counts = df_cat[col].value_counts().head(10)  # Pour éviter d'avoir trop de valeurs
    fig.add_trace(go.Bar(x=value_counts.index, y=value_counts.values, name=col), row=1, col=i+1)


fig.update_layout(height=700, width=1300, title="Fréquence des valeurs uniques par colonne (category)", showlegend=False)

fig.show()

In [ ]:
# Répartition des valeurs pour chaque colonne (graphique)
columns = df_num.columns
titles = ["Size", "n_pkt_src", "n_pkt_dst"]

fig = sp.make_subplots(rows=2, cols=2, subplot_titles=titles, vertical_spacing=0.1, horizontal_spacing=0.15)
for i, col in enumerate(columns):
    row = (i // 2) + 1
    col_position = (i % 2) + 1
    fig.add_trace(Bar(x=df_num[col].value_counts().index, y=df_num[col].value_counts().values, name=titles[i]),row=row, col=col_position)

fig.update_layout(height=700, width=700, title_text="Distribution des valeurs dans les colonnes numériques",showlegend=False)

fig.show()

- sport
    - Distribution : Forte concentration de valeurs spécifiques (port 502 par exemple qui est spécifique aux Modbus), certains ports moins représentés pourrait être des accès interdits

- dport
    - Comme pour sport, la majorité des paquets sont concentrés sur quelques ports de destination, avec le port 502 comme destination principale. \
    Des valeurs plus rares sont observées, mais elles sont peu fréquentes. Cela pourrait également être des attaques.

- flags
    - La plupart des valeurs sont à 11000, forte concentration autour de valeurs spécifiques (états standards de connexion ??)
    - La majorité des paquets ont des flags identiques, ce qui est attendu pour des communications stables. \
    Quelques variations existent, ce qui peut représenter des tentatives d’intrusion ou des anomalies.

- size
    - Tailles de paquets concentrées autour de valeurs entre 64 et 66 octets + peu de variation
    - Distribution fortement centrée sur 66 octets (taille de paquets standard pour les échanges de données Modbus ??) \
    --> paquets d'autres tailles pourraient être intéressants à analyser pour des comportements anormaux

- n_pkt_src
    - La distribution présente des pics élevés pour certaines valeurs spécifiques (flux réguliers de données ?? surcharge ou un potentiel DoS ???)

- n_pkt_dst
    - Pics dans la distribution indiquent des destinations qui reçoivent un grand nombre de paquets ?? anomalies ??

- label_n
    - Valeurs sont binaires (0 ou 1), avec 0 = paquets normaux et 1 attaque
    -La majorité des paquets sont normaux, donc dataset déséquilibré MAIS représentatif ?

### Analyse des colonnes catégorielles

In [ ]:
# Valeurs uniques pour chaque colonne
for col in df_cat.columns:
    print(col, df_cat[col].value_counts())

In [ ]:
fig = make_subplots(rows=1, cols=len(df_cat.columns), subplot_titles=df_cat.columns)

for i, col in enumerate(df_cat.columns):
    value_counts = df_cat[col].value_counts().head(10)  # Pour éviter d'avoir trop de valeurs
    fig.add_trace(go.Bar(x=value_counts.index, y=value_counts.values, name=col), row=1, col=i+1)


fig.update_layout(height=700, width=1300, title="Fréquence des valeurs uniques par colonne (category)", showlegend=False)

fig.show()

### Analyse des colonnes de type objet

In [ ]:
# Valeurs uniques pour chaque colonne
for col in df_obj.columns:
    print(col, df_obj[col].value_counts())

In [ ]:
fig = make_subplots(rows=1, cols=len(df_obj.columns), subplot_titles=df_obj.columns)

for i, col in enumerate(df_obj.columns):
    value_counts = df_obj[col].value_counts().head(10)  # Pour éviter d'avoir trop de valeurs
    fig.add_trace(go.Bar(x=value_counts.index, y=value_counts.values, name=col), row=1, col=i+1)


fig.update_layout(height=700, width=1300, title="Fréquence des valeurs uniques par colonne (object)", showlegend=False)

fig.show()

- mac_s, mac_d : 
    - une adresse principale largement dominante 
    - trafic centré autour de quelques appareils
    - autres sont peut-être anormales

- ip_s, ip_d : 
    - adresses IP source et destination également dominées par quelques valeurs
    - communication entre des hôtes spécifiques

- proto : 
    - protocole Modbus de loin le plus utilisé,
    - confirme l’utilisation majoritaire de Modbus dans le système

- modbus_fn : 
    - Les fonctions Modbus, toutes utilisées de façon plus ou moins équivalentes
    - Quelques valeurs inconnues, peut-être attaques

- modbus_response : 
    - réponse "Pas de réponse" dominante, surement représentant une exécution correcte
    - autres valeurs beaucoup moins présentes

- label : 
    - majorité des données marquées comme normales
    - autres valeurs non normales

## Matrice de corrélation

### Matrice dans le dataset normal

In [ ]:
df = df_net_norm

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset normal", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

Nous pouvons voir que ip_s, sport et n_pkt_src sont très corrélés, de même pour ip_d, dport et n_pkt_dst. \
Il y a également une forte corrélation négative entre les ports/ip sources et ceux de destination \
Cela semble plutot logique, car les échanges sont normalement entre les mêmes appareils.

In [ ]:
df = df_net_1

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset attack 1", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

Peu de différence avec le dataset normale, il va être compliqué de détecter des anomalies dans ce dataset. \
Surement dû au fait que les attaques sont physique en grande partie dans ce dataset, et donc non détectable sur des données réseaux.

In [ ]:
df = df_net_2

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset attack 2", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

In [ ]:
df = df_net_3

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset attack 3", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

In [ ]:
df = df_net_4

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset attack 4", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

Ces 3 dataset ont des matrices de corrélations assez différentes du dataset normale. \
Ces dataset ont des attaques physiques en moindre quantités comme vu précédemment (notebook_network_preparation.ipynb), il est donc plus facile de détecter les attaques qui sont visibles sur les données réseau.

In [ ]:
# Supposons que 'ecart' est une série Pandas contenant les écarts de temps en secondes
ecart_filtered = ecart.dropna()  # Supprimer les valeurs manquantes

# Créer un histogramme avec Plotly
fig = px.histogram(
    ecart_filtered, 
    nbins=1000,  # Nombre de bins
    title='Distribution des écarts de temps',
    labels={'value': 'Écarts de temps (secondes)', 'count': 'Fréquence'}
)

# Ajuster les limites de l'axe X
fig.update_xaxes(range=[0, 0.004])

# Personnalisation du graphique
fig.update_layout(
    xaxis_title="Écarts de temps (secondes)",
    yaxis_title="Fréquence"
)

# Afficher le graphique
fig.show()

L'acquisition des données n'est pas à la même fréquence que celle des données physiques. En effet, les données physiques sont enregistrées toutes les 1s, tandis qu'ici, il n'y a pas d'enregistrement continue. 

Les données sont enregistrées lorsqu'il y a une interraction réseau.